In [1]:
!pip -q install datasets transformers evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platfo

In [2]:
import re
import torch
from datasets import Dataset, load_dataset
from transformers import (
  DistilBertTokenizerFast,
  DistilBertForSequenceClassification,
  Trainer,
  TrainingArguments,
)

In [3]:
GSM8K_SPLIT    = "main"                     # HF split name for GSM8K
DEVICE         = "cuda" if torch.cuda.is_available() else "cpu"
NEG_PER_POS    = 1                          # # of corrupted negatives per correct step
MAX_LEN_STEP   = 64
EPOCHS         = 3
BATCH_SIZE     = 16
LEARNING_RATE  = 5e-5
TEST_SIZE      = 0.1
SEED           = 42
OUTPUT_DIR     = "verifier_ckpt_from_answer"

In [4]:
def corrupt_line(line: str) -> str:
  m = re.search(r"(\d+)\s*([\+\-\*/])\s*(\d+)", line)

  if m:
    a, op, b = m.groups()
    a, b = int(a), int(b)

    # flip + ↔ - or * ↔ /
    if op in "+-":
      op2 = "-" if op == "+" else "+"
    else:
      op2 = "*" if op == "/" else "/"

    return line.replace(f"{a}{op}{b}", f"{a}{op2}{b}", 1)

  # fallback: remove first digit
  return re.sub(r"\d", "", line, count=1)

In [5]:
print("Loading GSM8K train split…")

raw = load_dataset("gsm8k", GSM8K_SPLIT, split="train")

texts, labels = [], []

print("Extracting steps and building positives/negatives…")

for ex in raw:
  cot_answer = ex["answer"].strip()   # assumed to contain full CoT + final answer

  # split on periods
  steps = [s.strip() for s in cot_answer.split(".") if s.strip()]

  for step in steps:

    # positive example
    texts.append(step)
    labels.append(1)

    # generate NEG_PER_POS corrupted negatives
    for _ in range(NEG_PER_POS):
      texts.append(corrupt_line(step))
      labels.append(0)

Loading GSM8K train split…


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

Extracting steps and building positives/negatives…


In [6]:
print("Creating HuggingFace Dataset…")
ds = Dataset.from_dict({"text": texts, "label": labels})
ds = ds.train_test_split(test_size=TEST_SIZE, seed=SEED)

Creating HuggingFace Dataset…


In [7]:
print("Tokenizing examples…")

tok = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

def tokenize_fn(batch):
  return tok(
    batch["text"],
    truncation=True,
    padding="max_length",
    max_length=MAX_LEN_STEP
  )

ds = ds.map(tokenize_fn, batched=True)
ds.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Tokenizing examples…


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/53890 [00:00<?, ? examples/s]

Map:   0%|          | 0/5988 [00:00<?, ? examples/s]

In [8]:
print("Initializing DistilBERT classifier…")

model = DistilBertForSequenceClassification.from_pretrained(
  "distilbert-base-uncased",
  num_labels=2
).to(DEVICE)

args = TrainingArguments(
  output_dir=OUTPUT_DIR,
  num_train_epochs=EPOCHS,
  per_device_train_batch_size=BATCH_SIZE,
  per_device_eval_batch_size=BATCH_SIZE * 2,
  learning_rate=LEARNING_RATE,
  eval_strategy="epoch",
  save_strategy="epoch",
  logging_steps=100,
  load_best_model_at_end=True,
  metric_for_best_model="accuracy",
  seed=SEED,
  report_to= "none",
)

from evaluate import load

accuracy = load("accuracy")

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  preds = logits.argmax(-1)
  return accuracy.compute(predictions=preds, references=labels)

trainer = Trainer(
  model=model,
  args=args,
  train_dataset=ds["train"],
  eval_dataset=ds["test"],
  compute_metrics=compute_metrics,
)

Initializing DistilBERT classifier…


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
print("Training verifier…")
trainer.train()

print(f"Saving verifier and tokenizer to `{OUTPUT_DIR}`...")
trainer.save_model(OUTPUT_DIR)
tok.save_pretrained(
  OUTPUT_DIR,
  safe_serialization=False
)

print("Verifier training complete.")

Training verifier…


Epoch,Training Loss,Validation Loss,Accuracy
1,0.218200,0.198286,0.906313
2,0.183600,0.195181,0.911156
3,0.172600,0.201989,0.912826


Saving verifier and tokenizer to `verifier_ckpt_from_answer`...
Verifier training complete.
